# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('/Users/soul/Desktop/IronHack/lab-intro-recommender-systems/data/customer_product_sales.csv')

In [3]:
data.head()

CustomerID FirstName   LastName  SalesID  ProductID           ProductName  \
0       61288      Rosa   Andersen   134196        229  Bread - Hot Dog Buns   
1       77352     Myron     Murray  6167892        229  Bread - Hot Dog Buns   
2       40094     Susan  Stevenson  5970885        229  Bread - Hot Dog Buns   
3       23548    Tricia    Vincent  6426954        229  Bread - Hot Dog Buns   
4       78981     Scott      Burch   819094        229  Bread - Hot Dog Buns   

   Quantity  
0        16  
1        20  
2        11  
3         6  
4        20

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [155]:
df = data.groupby(['CustomerID', 'ProductName']).sum()[['Quantity']]
df.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [147]:
df_pivot = df.pivot_table(index='ProductName', columns='CustomerID', values='Quantity').fillna(0)
df_pivot.head()

CustomerID                         33     200    264    356    412    464    \
ProductName                                                                   
Anchovy Paste - 56 G Tube            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mushroom Tart            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0    0.0    0.0    0.0   
Apricots - Dried                     1.0    0.0    0.0    0.0    1.0    0.0   

CustomerID                         477    639    649    669    ...  97697  \
ProductName                                                    ...          
Anchovy Paste - 56 G Tube            0.0    1.0    0.0    0.0  ...    0.0   
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0    0.0  ...   25.0   
Appetizer - Mushroom Tart            0.0    1.0    0.0    0.0  ...   25.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0    0.0  ...    0.0   
Apricots - Dried                     0.0    0.0    0.0    0.0  ...    0.0   

CustomerID                         97753  97769  97793  97900  97928  98069  \
ProductName                                                                   
Anchovy Paste - 56 G Tube           25.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mini Egg Roll, Shrimp   25.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mushroom Tart            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0   25.0   25.0   25.0   
Apricots - Dried                    25.0    0.0    0.0    0.0    0.0    0.0   

CustomerID                         98159  98185  98200  
ProductName                                             
Anchovy Paste - 56 G Tube            0.0    0.0    0.0  
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0  
Appetizer - Mushroom Tart            0.0   25.0    0.0  
Appetizer - Sausage Rolls            0.0   25.0    0.0  
Apricots - Dried                     0.0    0.0    0.0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [140]:
similarity_matrix = squareform(pdist(df_pivot.T, 'euclidean'))
similarity_matrix

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [152]:
similarity_df = pd.DataFrame(similarity_matrix, index=df_pivot.columns, columns=df_pivot.columns)
similarity_index = similarity_df[264].sort_values()[1:6].index.tolist()


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [51]:
similar_prods = df.loc[similarity_index]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [56]:
ranked_products = similar_prods.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [106]:
ranked_and_customer = ranked_products.merge(df_pivot, left_index=True, right_index=True)
top5 = ranked_and_customer[['Quantity',264]][ranked_and_customer[264] == 0.0].sort_values(by='Quantity', ascending = False).head()
recom_prods = top5.index.tolist()

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [153]:
recommendations = {}
unique_custID = list(data.CustomerID.unique())

for customer in unique_custID:
    similarity_index = similarity_df[customer].sort_values()[1:6].index.tolist()
    similar_prods = df.loc[similarity_index]
    ranked_products = similar_prods.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)
    ranked_and_customer = ranked_products.merge(df_pivot, left_index=True, right_index=True)
    top5 = ranked_and_customer[['Quantity', customer]][ranked_and_customer[customer] == 0.0].sort_values(by='Quantity', ascending = False).head()
    recom_prods = top5.index.tolist()
    recommendations[customer] = recom_prods
    

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [119]:
recommendations_df = pd.DataFrame(recommendations)
recommendations_df.head()

61288                     77352  \
0    Beef - Rib Eye Aaa  Wine - Valpolicella Masi   
1   Chicken - Soup Base              Tahini Paste   
2          Jagermeister    Ocean Spray - Ruby Red   
3  Lime Cordial - Roses               Fuji Apples   
4  Lamb - Pieces, Diced       Tuna - Salad Premix   

                            40094                            23548  \
0  Bread - Italian Corn Meal Poly          Tea - English Breakfast   
1                   Puree - Mocha  Squid - Tubes / Tenticles 10/20   
2     Wine - White, Colubia Cresh       Bread - Raisin Walnut Oval   
3            Cheese - Brie,danish                           Pernod   
4          Cookie - Dough Variety                      Wanton Wrap   

                78981                             83106  \
0             Isomalt  Cheese - Boursin, Garlic / Herbs   
1  Crush - Cream Soda                 Wiberg Super Cure   
2       Vanilla Beans          Beef - Top Sirloin - Aaa   
3              Cassis                   Garlic - Peeled   
4     Tea - Earl Grey            Cake - Mini Cheesecake   

                             11253                     35107  \
0                     Juice - Lime          Vinegar - Sherry   
1      Pork - Bacon, Double Smoked  Table Cloth 62x114 White   
2               Tomatoes Tear Drop   Bread - French Baquette   
3                          Pomello     Spoon - Soup, Plastic   
4  Soup - Campbells Tomato Ravioli        Lentils - Red, Dry   

                         15088                    26031  ...  \
0  Soup Knorr Chili With Beans             Sugar - Fine  ...   
1      Soupfoamcont12oz 112con      Cheese - Mozzarella  ...   
2          Beer - Rickards Red  Pork - Loin, Center Cut  ...   
3          Arizona - Green Tea            Hersey Shakes  ...   
4             Eggplant - Asian    Pop Shoppe Cream Soda  ...   

                         25270                        17390  \
0               Black Currants   Pasta - Penne, Rigate, Dry   
1           Beef - Top Sirloin  Soup Knorr Chili With Beans   
2           Onions - Cippolini      Coffee - Hazelnut Cream   
3  Soup Knorr Chili With Beans             Scallops - 10/20   
4             Eggplant - Asian                     Rambutan   

                   97029                        38154  \
0         Fenngreek Seed                  Milk Powder   
1               Bay Leaf      Rum - Mount Gay Eclipes   
2  Veal - Inside, Choice             Sea Bass - Whole   
3                Pomello                   Watercress   
4         Banana Turning  Cake - Box Window 10x10x2.5   

                           74403                       49005  \
0  Chips Potato Salt Vinegar 43g          Pickerel - Fillets   
1             Tomatoes Tear Drop                Phyllo Dough   
2                 Fenngreek Seed  Pasta - Penne, Rigate, Dry   
3                        Isomalt      Browning Caramel Glace   
4                       Bay Leaf   Ecolab - Mikroklene 4/4 L   

                   41286                     85878  \
0   Beef - Ground Medium    Cake - Mini Cheesecake   
1     Beer - Labatt Blue         Olives - Kalamata   
2   Thyme - Lemon, Fresh  Wine - White Cab Sauv.on   
3        Wonton Wrappers             Yoghurt Tubes   
4  Veal - Inside, Choice       Veal - Eye Of Round   

                          68506                          3472   
0            Onions - Cippolini          Pepper - Black, Whole  
1          Beef - Ground Medium              Butter - Unsalted  
2                   Scampi Tail  Wine - Ej Gallo Sierra Valley  
3  Veal - Brisket, Provimi,bnls                Beef Wellington  
4    Rosemary - Primerba, Paste          Knife Plastic - White  

[5 rows x 1000 columns]

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [151]:
similarity_matrix_corr = squareform(pdist(df_pivot.T, 'correlation'))
similarity_df = pd.DataFrame(similarity_matrix_corr, index=df_pivot.columns, columns=df_pivot.columns)

recom = {}
for customer in unique_custID:
    similarity_index = similarity_df[customer].sort_values()[1:6].index.tolist()
    similar_prods = df.loc[similarity_index]
    ranked_products = similar_prods.groupby('ProductName').sum().sort_values(by='Quantity', ascending=False)
    ranked_and_customer = ranked_products.merge(df_pivot, left_index=True, right_index=True)
    top5 = ranked_and_customer[['Quantity', customer]][ranked_and_customer[customer] == 0.0].sort_values(by='Quantity', ascending = False).head()
    recom_prods = top5.index.tolist()
    recom[customer] = recom_prods

In [148]:
similarity_matrix.max()

330.1635958127425

In [154]:
similarity_matrix_corr.max()

1.1741772972276645

In [139]:
recom

{61288: ['Jagermeister',
  'Wine - Two Oceans Cabernet',
  'Tofu - Firm',
  'Appetizer - Sausage Rolls',
  'Dc - Frozen Momji'],
 77352: ['Fondant - Icing',
  'Grenadine',
  'Mangoes',
  'Veal - Osso Bucco',
  'Flour - Whole Wheat'],
 40094: ['Bread - Roll, Soft White Round',
  'Truffle Cups - Brown',
  'Juice - V8 Splash',
  'Veal - Brisket, Provimi,bnls',
  'Anchovy Paste - 56 G Tube'],
 23548: ['Olives - Stuffed',
  'Wine - Charddonnay Errazuriz',
  'Assorted Desserts',
  'Soup - Campbells Tomato Ravioli',
  'Juice - Orange'],
 78981: ['Tart Shells - Sweet, 4',
  'Sprouts - Baby Pea Tendrils',
  'Artichokes - Jerusalem',
  'Chocolate - Dark',
  'Appetizer - Mini Egg Roll, Shrimp'],
 83106: ['Pastry - Cheese Baked Scones',
  'Bouq All Italian - Primerba',
  'Onions - Vidalia',
  'Bread - Italian Roll With Herbs',
  'Turnip - White, Organic'],
 11253: ['Sauce - Demi Glace',
  'Nut - Chestnuts, Whole',
  'Beef - Tenderlion, Center Cut',
  'Mushroom - Trumpet, Dry',
  'Soup - Canadian P

In [142]:
recommendations

{61288: ['Jagermeister',
  'Wine - Two Oceans Cabernet',
  'Tofu - Firm',
  'Appetizer - Sausage Rolls',
  'Dc - Frozen Momji'],
 77352: ['Fondant - Icing',
  'Grenadine',
  'Mangoes',
  'Veal - Osso Bucco',
  'Flour - Whole Wheat'],
 40094: ['Bread - Roll, Soft White Round',
  'Truffle Cups - Brown',
  'Juice - V8 Splash',
  'Veal - Brisket, Provimi,bnls',
  'Anchovy Paste - 56 G Tube'],
 23548: ['Olives - Stuffed',
  'Wine - Charddonnay Errazuriz',
  'Assorted Desserts',
  'Soup - Campbells Tomato Ravioli',
  'Juice - Orange'],
 78981: ['Tart Shells - Sweet, 4',
  'Sprouts - Baby Pea Tendrils',
  'Artichokes - Jerusalem',
  'Chocolate - Dark',
  'Appetizer - Mini Egg Roll, Shrimp'],
 83106: ['Pastry - Cheese Baked Scones',
  'Bouq All Italian - Primerba',
  'Onions - Vidalia',
  'Bread - Italian Roll With Herbs',
  'Turnip - White, Organic'],
 11253: ['Sauce - Demi Glace',
  'Nut - Chestnuts, Whole',
  'Beef - Tenderlion, Center Cut',
  'Mushroom - Trumpet, Dry',
  'Soup - Canadian P